In [1]:
from keras.models import load_model
import keras
import numpy as np
from h5py import File
from juliacall import Main as jl
import matplotlib.pyplot as plt
jl.seval("using BayesianTomography")

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
trials = 1

final_fids = np.empty(trials, dtype="float32")
final_stds = np.empty(trials, dtype="float32")

orders = range(5,6)

for m in range(len(orders)):
    order = orders[m]

    with File('../Data/Processed/fixed_order_intense.h5') as f:
        direct = keras.layers.Resizing(64, 64)(
            f[f'images_order{order}'][:, 0, :, :])
        converted = keras.layers.Resizing(64, 64)(
            f[f'images_order{order}'][:, 1, :, :])
        rho = np.conj(f[f'labels_order{order}'][:])

    x = np.stack((direct, converted), axis=1)
    mu = x.mean(axis=(-1, -2), keepdims=True)
    sigma = x.std(axis=(-1, -2), keepdims=True)
    x = (x - mu) / sigma

    fids = np.empty((trials, rho.shape[0]), dtype="float32")

    for m in range(trials):
        trial = m+1
        model = load_model(
            f"TrainedModels/FixedOrderIntense/order{order}_trial{trial}.keras")

        y_pred = np.array(model(x))

        for n in np.arange(fids.shape[1]):
            rho_pred = jl.density_matrix_reconstruction(y_pred[n])
            fids[m, n] = jl.fidelity(rho[n], rho_pred)

    mean_fids = fids.mean(axis=1)
    j = mean_fids.argmax()

    final_fids[m] = mean_fids[j]
    final_stds[m] = fids.std(axis=1)[j]

In [3]:
for m in range(len(orders)):
    order = orders[m]
    print(f'Order {order}:', final_fids[m], '+/-', final_stds[m])

Order 5: 0.7747734 +/- 0.10758743
